<a href="https://colab.research.google.com/github/AndrewstheBuilder/grokking_deeplearning/blob/main/RNN_CH12_Grokking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start Linear RNN

In [1]:
! wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-1.tar.gz
! tar -xvf tasks_1-20_v1-1.tar.gz


--2024-03-23 22:33:44--  http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-1.tar.gz
Resolving www.thespermwhale.com (www.thespermwhale.com)... 50.31.160.191
Connecting to www.thespermwhale.com (www.thespermwhale.com)|50.31.160.191|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1282454 (1.2M) [application/x-gzip]
Saving to: ‘tasks_1-20_v1-1.tar.gz’

tasks_1-20_v1-1.tar 100%[===================>]   1.22M  5.24MB/s    in 0.2s    

2024-03-23 22:33:45 (5.24 MB/s) - ‘tasks_1-20_v1-1.tar.gz’ saved [1282454/1282454]

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/

Read in Data

In [2]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('tasksv11/en/qa1_single-supporting-fact_train.txt', 'r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
  tokens.append(line.lower().replace("\n", "").split(" ")[1:])

print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


Define Util functions and util data structures

In [11]:
# Few useful counts, lists, and utility functions to use during predict, compare, and learn process
vocab = set()
for sent in tokens:
  for word in sent:
    # print('word',word)
    vocab.add(word)

vocab = list(vocab)
# print('vocab',vocab)
word2index = {}
for i,word in enumerate(vocab):
  # print('i',i)
  # print('word',word)
  word2index[word]=i

def words2indices(sentence):
  idx = list()
  for word in sentence:
    idx.append(word2index[word])
  return idx

def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=0)
# print(word2index)
# print(tokens)

Initialize Parameters

In [26]:
np.random.seed(1)
embed_size = 10

# Word Embeddings
# Why do we scale it by subtracting 0.5 then multiplying by 0.1
# I do not get the order of doing this and why even do it?
embed = (np.random.rand(len(vocab), embed_size) - 0.5) * 0.1

# Embedding -> Embedding(initially the identity matrix)
recurrent = np.eye(embed_size)

# sentence embedding for an empty sentence
start = np.zeros(embed_size)

# Embedding -> output weights
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1

# One hot lookups (for the loss function)
# Where does this even come from??
# And why does the book not go into it?
one_hot = np.eye(len(vocab))

Define Predict Function

In [27]:
def predict(sent):
  # print('sent',sent)
  layers = list()
  layer = {}
  layer['hidden'] = start
  # print('layer["hidden"]',layer['hidden'])
  layers.append(layer)

  loss = 0

  for target_i in range(len(sent)):
    layer = {}
    # Tries to predict the next term
    # The decoder is used to translate the result from the hidden layer to dimensions that the softmax will understand
    # Does the decoder get trained? --> Yes
    # *TODO: What if I do not train the decoder?
    layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))
     # What is the probability that RNN predicted sent[target_i]
    loss += -np.log(layer['pred'][sent[target_i]])
    # Generates the next hidden state
    # What is the reasoning behind how the next hidden state gets generated?
    layer['hidden'] = layers[-1]['hidden'].dot(recurrent) +\
                        embed[sent[target_i]]
    layers.append(layer)

  return layers, loss

Weight Updates

In [29]:
# forward
iterations = 10000
for iter in range(iterations):
    alpha = 0.001
    # print('tokens[iter]',tokens[iter])
    # print('tokens[iter][1:]',tokens[iter][1:])
    # print('len(tokens)',len(tokens))
    # sent1 = words2indices(tokens[iter%len(tokens)])
    sent = words2indices(tokens[iter%len(tokens)][1:])
    # print('sent',sent)
    # print('sent1',sent1)

    layers,loss = predict(sent)

    # back propagate
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a
            # later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else:
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

    # update weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx,layer in enumerate(layers[1:]):

        # TODO: Turned off decoder learning here!
        # decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))

        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))

    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))

Perplexity:81.06181320165963
Perplexity:80.90945686888239
Perplexity:80.73439256091658
Perplexity:80.53003733224655
Perplexity:80.28735480087705
Perplexity:79.99364227628391
Perplexity:79.63059726830139
Perplexity:79.17119966840244
Perplexity:78.57473210102954
Perplexity:77.77934127761092


In [30]:
sent_index = 100

l,_ = predict(words2indices(tokens[sent_index]))

print(tokens[sent_index])

for i,each_layer in enumerate(l[1:-1]):
    input = tokens[sent_index][i]
    true = tokens[sent_index][i+1]
    pred = vocab[each_layer['pred'].argmax()]
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

['john', 'moved', 'to', 'the', 'kitchen.']
Prev Input:john        True:moved          Pred:	bathroom	1
Prev Input:moved       True:to             Pred:	bathroom	1
Prev Input:to          True:the            Pred:kitchen.
Prev Input:the         True:kitchen.       Pred:kitchen.
